# Gemini as a Customer Service Agent demo 

## Email Generation

### Generate customer personas from user activity

Start by using Gemini and your BigQuery data to generate customer personas based on the customer's purchasing frequency, recency, and average spend.

In [ ]:
%%bigquery results
CREATE OR REPLACE TABLE `data-quality-demo-next-24.cymbal_sports_marketing.customer_personas`
AS
with hold AS(
  SELECT centroid AS cluster, ml_generate_text_llm_result AS persona
  FROM ML.GENERATE_TEXT(
    MODEL `data-quality-demo-next-24.cymbal_sports.generate_text`,
    (
      with clusters AS(
        SELECT
        centroid_id as centroid,
        avg_spend as average_spend,
        count_orders as count_of_orders,
        days_since_order
        FROM (
          SELECT centroid_id, feature, ROUND(numerical_value, 2) as value
          FROM ML.CENTROIDS(MODEL `data-quality-demo-next-24.cymbal_sports_marketing.customer_segment_clustering`)
        )
        PIVOT (
          SUM(value)
          FOR feature IN ('avg_spend',  'count_orders', 'days_since_order')
        )
        ORDER BY centroid_id
      )

      SELECT
        'Provide the following response in a JSON format. Pretend you are a creative strategist, given the following clusters come up with a creative persona_title and persona_description of these personas for each of these clusters' || ' ' || clusters.centroid || ', Average Spend $' || clusters.average_spend || ', Count of orders per person ' || clusters.count_of_orders || ', Days since last order ' || clusters.days_since_order || 'The parent fields are persona_title and persona_description. Parent fields should be lower cased. There are no child fields. Do not include JSON decorator. This is an example of an output that is correctly formatted:{{"persona_title": "The Frequent Flyer", "response_email": "A loyal customer who makes multiple orders per year. They tend to order frequently and spend a lot of money when they do order. They are highly engaged with our brand."}}' AS prompt,
        centroid
      FROM
        clusters
    ),
    -- See the BigQuery 'Generate Text' docs to better understand how changing these inputs will impact your results: https://cloud.google.com/bigquery/docs/generate-text#generate_text
      STRUCT(
        800 AS max_output_tokens,
        0.8 AS temperature,
        40 AS top_k,
        0.8 AS top_p,
        TRUE AS flatten_json_output
      )
    )
)
SELECT
  cluster, TRIM(JSON_QUERY(persona, '$.persona_title'),'"') as persona_title, TRIM(JSON_QUERY(persona, '$.persona_description'),'"') as persona_description
  FROM hold
;


### BigQuery's `ML.GENERATE_TEXT` Function

BigQuery can call Gemini using the [`ML.GENERATE_TEXT`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-text) Table Value Function (TVF). To learn more about how you can generate text using your own data as inputs, check out [the guide](https://cloud.google.com/bigquery/docs/generate-text) in our documentation.

Here we'll call a stored procedure with the saved query to invoke Gemini. This will take our user data to identify the appropriate persona for an individual user based on their purchasing behavior, join in key product recommendations, and generate a customized email for that person.

In [ ]:
%%bigquery results
CALL `data-quality-demo-next-24.cymbal_sports_marketing.sp_generate_email`();

In [ ]:
results